<a href="https://colab.research.google.com/github/ooutaksaluk/MBT-analysis/blob/main/Main_Battle_Tank_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Main Battle Tank analysis**



In [1]:
!pip install anvil-uplink opencv-python numpy torch torchvision

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

train_dir = '/content/drive/My Drive/dataset/train'
validation_dir = '/content/drive/My Drive/dataset/validation'

# ตรวจสอบว่าโฟลเดอร์มีอยู่จริง
if os.path.exists(train_dir):
    print("พบโฟลเดอร์:", train_dir)
else:
    print("ไม่พบโฟลเดอร์:", train_dir)


พบโฟลเดอร์: /content/drive/My Drive/dataset/train


**เตรียมข้อมูลและการโหลดข้อมูลด้วย Keras**

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # ใช้ 20% เป็น validation
)

# โหลดข้อมูล Train (ใช้ 80%)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# โหลดข้อมูล Validation (ใช้ 20%)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)



Found 72 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


**Training Model**

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# สร้างโมเดล CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')  # 2 Output: Sight, Mantlet
])

# ใช้ binary_crossentropy เพราะเป็น Multi-Label (sigmoid)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.save('/content/drive/My Drive/dataset/trained_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
for layer in model.layers:
    print(layer.name)

conv2d
max_pooling2d
conv2d_1
max_pooling2d_1
conv2d_2
max_pooling2d_2
flatten
dense
dropout
dense_1


In [24]:
print("🔍 เลเยอร์ Conv2D สุดท้าย:", LAST_CONV_LAYER)


🔍 เลเยอร์ Conv2D สุดท้าย: conv2d_2


In [18]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
import tensorflow as tf



# กำหนดชื่อเลเยอร์ Convolution สุดท้าย
LAST_CONV_LAYER = "conv2d_2"

def make_gradcam_heatmap(model, img_array, last_conv_layer_name):
    """ สร้าง Heatmap จาก Grad-CAM """
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

def overlay_heatmap(img, heatmap, alpha=0.4):
    """ ซ้อน Heatmap ลงบนภาพ """
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return superimposed_img




**Anvil**



In [36]:
import anvil.server
import anvil.media
import tensorflow as tf
import numpy as np
import cv2

# กำหนด Input Shape
INPUT_SHAPE = (150, 150, 3)

# โหลดโมเดล
model = tf.keras.models.load_model('/content/drive/My Drive/dataset/trained_model.h5')

#  บังคับให้โมเดลรับอินพุตก่อน
dummy_input = tf.zeros((1, 150, 150, 3))
_ = model(dummy_input, training=False)

#  ตอนนี้สามารถใช้ model.input ได้อย่างปลอดภัย
grad_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[model.get_layer(last_conv_layer_name).output, model.output]
)

# หาเลเยอร์ Conv2D สุดท้าย
LAST_CONV_LAYER = None
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        LAST_CONV_LAYER = layer.name
        break

if LAST_CONV_LAYER is None:
    raise ValueError("ไม่พบเลเยอร์ Conv2D ในโมเดล!")

print(f"Conv2D สุดท้ายที่ใช้: {LAST_CONV_LAYER}")

def make_gradcam_heatmap(model, img_array, last_conv_layer_name):
    """สร้าง Heatmap จาก Grad-CAM"""
    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array, training=False)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)

    if np.max(heatmap) > 0:
        heatmap /= np.max(heatmap)  # Normalize

    return heatmap

@anvil.server.callable
def process_image(file):
    print("process_image() ถูกเรียกแล้ว!")
    print(f"ไฟล์ที่ได้รับ: {file.content_type}, ขนาด: {len(file.get_bytes())} Bytes")

    # โหลดไฟล์เป็น OpenCV image
    file_bytes = file.get_bytes()
    nparr = np.frombuffer(file_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if img is None:
        print("ไม่สามารถโหลดรูปภาพได้!")
        return None

    print("โหลดภาพสำเร็จ กำลังประมวลผล...")

    # แปลงภาพให้เข้ากับโมเดล
    img_resized = cv2.resize(img, (150, 150))
    img_array = img_resized.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # ตรวจสอบการใช้โมเดล
    try:
        prediction = model.predict(img_array)[0]
        print("โมเดลประมวลผลสำเร็จ!")
    except Exception as e:
        print(f"เกิดข้อผิดพลาดกับโมเดล: {e}")
        return None

    # คำนวณ Heatmap
    heatmap = make_gradcam_heatmap(model, img_array, LAST_CONV_LAYER)

    # วาด Bounding Box ถ้ามีการตรวจจับ
    x, y, w, h = 10, 10, 50, 50  # ค่าเริ่มต้น (สมมติ)
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, "Detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # ส่งผลลัพธ์กลับไปยัง Anvil
    _, img_encoded = cv2.imencode('.jpg', img)
    return anvil.BlobMedia("image/jpeg", img_encoded.tobytes())

# เริ่มเซิร์ฟเวอร์ Anvil
anvil.server.wait_forever()


Conv2D สุดท้ายที่ใช้: conv2d_2
process_image() ถูกเรียกแล้ว!
ไฟล์ที่ได้รับ: image/jpeg, ขนาด: 8368 Bytes
โหลดภาพสำเร็จ กำลังประมวลผล...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
โมเดลประมวลผลสำเร็จ!


KeyboardInterrupt: 